[accounts](./accounts.ipynb) | [orders](./orders.ipynb) | [trades](./trades.ipynb) | [positions](./positions.ipynb) | [historical](./historical.ipynb) | [streams](./streams.ipynb) | [errors](./exceptions.ipynb)

# Historical data

OANDA provides access to historical data. The *oandapyV20* has a class to access this data: *oandapyV20.endpoints.instruments.InstrumentsCandles*.

Lets give it a try and download some data for:
  + instrument: EUR_USD
  + granularity: H1
  + from: 2017-01-01T00:00:00

In [ ]:
import json
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from exampleauth import exampleauth

accountID, access_token = exampleauth.exampleAuth()
client = oandapyV20.API(access_token=access_token)

instrument = "EUR_USD"
params = {
    "from": "2017-01-01T00:00:00Z",
    "granularity": "H1",
    "count": 10,
    "price":"BA",
}
r = instruments.InstrumentsCandles(instrument=instrument, params=params)
response = client.request(r)
print("Request: {}  #candles received: {}".format(r, len(r.response.get('candles'))))
print(json.dumps(response, indent=2))


In [57]:
import json
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from exampleauth import exampleauth
import pandas as pd
import requests
from datetime import timezone, datetime
accountID, access_token = exampleauth.exampleAuth()
client = oandapyV20.API(access_token=access_token)
storageLocation = "Instruments/Forex/Oanda/"
barTimeframe = "D" 
assetsToDownload = ["AUDNZD","AUDUSD","AUDJPY","EURCHF","EURGBP","EURJPY","GBPCHF","GBPJPY","GBPUSD","NZDUSD","USDCAD","USDCHF","USDJPY","CADJPY","EURAUD","CHFJPY","EURCAD","AUDCAD","AUDCHF","CADCHF","EURNZD","GBPAUD","GBPCAD","GBPNZD","NZDCAD","NZDCHF","NZDJPY","EURSGD","USDSGD","GBPSGD","AUDSGD","SGDJPY","NZDSGD","CADSGD","CHFSGD"]
iteratorPos = 0 # Tracks position in list of symbols to download
assetListLen = len(assetsToDownload)
start_datetime = datetime(2017, 1, 1)
#print('date:',start_datetime)
while iteratorPos < assetListLen:
	instrument = assetsToDownload[iteratorPos]
	instrument= instrument[:3] + '_' + instrument[3:]
	#print(symbol+'_'+barTimeframe+'.csv')
	iteratorPos += 1
	fileName=storageLocation+instrument+'_'+barTimeframe+'.csv'
	Path = os.path.isfile(fileName)
	print(type(start_datetime))
	if Path==True:
		oldDF=pd.read_csv(fileName)
		start_datetime=(oldDF.iloc[-1,0])
		print(type(start_datetime))
		print(fileName, start_datetime)
	#retreive the data from Oanda
	#start_unix = int(start_datetime.replace(tzinfo=timezone.utc).timestamp())
	start_unix=datetime.strptime(start_datetime, '%Y-%m-%dT%H:%M:%S.000000Z')
	params = {
    	"from": start_unix,
    	"granularity": barTimeframe,
    	"count": 5000,
	}
	print(instrument)
	r = instruments.InstrumentsCandles(instrument=instrument, params=params)
	client.request(r)
	r.response['candles'][0]['mid']
	r.response['candles'][0]['time']
	r.response['candles'][0]['volume']
	dat = []
	for oo in r.response['candles']:
		dat.append([oo['time'], oo['mid']['o'], oo['mid']['h'], oo['mid']['l'], oo['mid']['c'], oo['volume'], oo['complete']])
		df = pd.DataFrame(dat)
		df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Complete']
		#print("DF ORIG:",df)
		#Convert To Float
		df["Time"] = pd.to_datetime(df["Time"], unit='ns')
		df["Open"] = pd.to_numeric(df["Open"], downcast="float")
		df["High"] = pd.to_numeric(df["High"], downcast="float")
		df["Low"] = pd.to_numeric(df["Low"], downcast="float")
		df["Close"] = pd.to_numeric(df["Close"], downcast="float")
		df["Time"] = pd.to_datetime(df["Time"], unit='ns')
		incompletDF=df[~df["Complete"]]
		df=df.drop(['Complete'], axis=1)
		
		# Verifies if symbol file exists
	print("DF after:",df)	
	if Path==True:
		exclude=pd.read_csv(fileName)
		df=df[~df.Time.isin(exclude.Time)]
		df.to_csv(fileName, mode='a',index=False, header=False)	
	else:
		df.to_csv(fileName,index=False)
	iteratorPos += 1
	break

<class 'datetime.datetime'>
<class 'str'>
Instruments/Forex/Oanda/AUD_NZD_D.csv 2022-02-10 22:00:00+00:00


TypeError: str.replace() takes no keyword arguments

So, that is 9 records?
... that can be fixed by including the parameter *includeFirst*, see the OANDA documentation for details.

In [ ]:
instrument = "EUR_USD"
params = {
    "from": "2017-01-01T00:00:00Z",
    "granularity": "H1",
    "includeFirst": True,
    "count": 10,
}
r = instruments.InstrumentsCandles(instrument=instrument, params=params)
response = client.request(r)
print("Request: {}  #candles received: {}".format(r, len(r.response.get('candles'))))
print(json.dumps(response, indent=2))

# Bulk history

## InstrumentsCandles class

It is likely that you want to retrieve more than 10 records. The OANDA docs say that the default number of records
is 500, in case you do not specify. You can specify the number of records to retrieve by using *count*, with a maximum of 5000. The *InstrumentsCandles* class enables you to retrieve the records.

## InstrumentsCandlesFactory

Now if you would like to retrieve a lot of history, you have to make consecutive requests. To make this an easy process the *oandapyV20* library comes with a so called *factory* named *InstrumentsCandlesFactory*.

Using this class you can retrieve all history of an instrument from a certain date. The *InstrumentsCandlesFactory* acts as a generator generating *InstrumentCandles* requests until all data is retrieved. The number of requests can be influenced by specifying *count*. Setting *count* to 5000 would generate a tenth of the requests vs. the default of 500.

Back to our example: lets make sure we request a lot of data, so we set the *granularity* to *M5* and leave the date at 2017-01-01T00:00:00. The will retrieve all records from that date up to today, because we did not specify the *to* parameter. 


In [ ]:
import json
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from exampleauth import exampleauth

accountID, access_token = exampleauth.exampleAuth()
client = oandapyV20.API(access_token=access_token)

instrument = "EUR_USD"
params = {
    "from": "2017-01-01T00:00:00Z",
    "granularity": "M5",
}

def cnv(r, h):
    # get all candles from the response and write them as a record to the filehandle h
    for candle in r.get('candles'):
        ctime = candle.get('time')[0:19]
        try:
            rec = "{time},{complete},{o},{h},{l},{c},{v}".format(
                time=ctime,
                complete=candle['complete'],
                o=candle['mid']['o'],
                h=candle['mid']['h'],
                l=candle['mid']['l'],
                c=candle['mid']['c'],
                v=candle['volume'],
            )
        except Exception as e:
            print(e, r)
        else:
            h.write(rec+"\n")

datafile = "/tmp/{}.{}.out".format(instrument, params['granularity'])
with open(datafile, "w") as O:
    n = 0
    for r in InstrumentsCandlesFactory(instrument=instrument, params=params):
        rv = client.request(r)
        cnt = len(r.response.get('candles'))
        print("REQUEST: {} {} {}, received: {}".format(r, r.__class__.__name__, r.params, cnt))
        n += cnt
        cnv(r.response, O)
    print("Check the datafile: {} under /tmp!, it contains {} records".format(datafile, n))

## ... that was easy ...

All request were made on the default of max. 500 records per request. With a granularity of *M5* this means that we have 288 records per day. The algorithm of the factory does not check on weekends or holidays. Therefore some request only return a part of the 500 requested records because there simply are no more records within the specified timespan.

If you want to decrease the number of requests and increase the number of records returned for a request, just specify *count* as a number higher than 500 and up to max. 5000.